# Assignment 2

#### First Name: Blai
#### Last Name: Ras


## 1. Load Data from JSON

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 19.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=ea71d5c0bc146ea1650f80c463724d74610282a982f6b9f16c0306d81dbe6819
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:

from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Twitter Analysis")\
.getOrCreate()


In [ ]:
!git clone https://github.com/rohit-nlp/BigDataCourseUB.git

Cloning into 'BigDataCourseUB'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 146 (delta 10), reused 38 (delta 6), pack-reused 101
Receiving objects: 100% (146/146), 104.52 MiB | 21.81 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
df_twitter = spark.read.json("BigDataCourseUB/assisgnment_2/corona_tweet_new.json")

In [ ]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [ ]:
### From the user nestec col select the following cols only id_str,followers_count,friends_count and created at 
# (2 points)
from pyspark.sql.functions import col, struct

#Looking at the teacher's output below I see that the user columns have been renamed with "user_*" and that even though we are asked to take the user column "friends_count",
#I cannot see it in the teacher's output. Nevertheless, I take it just in case. It also looks like we got rid of the column "location", but later on we need it.

df_twitter = df_twitter.withColumn("user", struct(
                            col("user.id_str").alias("user_id_str"),
                            col("user.followers_count").alias("user_followers_count"),
                            col("user.friends_count").alias("user_friends_count"),
                            col("user.created_at").alias("user_created_at")
                            )).select(["created_at","favorite_count","hashtags","id","in_reply_to_status_id","in_reply_to_user_id_str","location","reply_count","retweet_count","source",
                                       "user.user_id_str","user.user_followers_count","user.user_created_at","user.user_friends_count"])


In [ ]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- user_friends_count: long (nullable = true)



In [ ]:
# Print the total count of number of records in df_twitter(1 point)
df_twitter.count()

15894

In [ ]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regualr expresion and the group would be 1 for this regular expression.
#(4 points)
from pyspark.sql.functions import regexp_extract

df_twitter=df_twitter.withColumn("extracted_source",regexp_extract('source',r'<a [^>]+>([^<]+)',1))
df_twitter.select(col('extracted_source'),col('source')).show()

+-------------------+--------------------+
|   extracted_source|              source|
+-------------------+--------------------+
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter Web Client|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
+----------

In [ ]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter.rdd.map(tuple)

In [ ]:
# Create a temporay table in memory with name as twitter (1 point)

df_twitter.createTempView("twitter")

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL 

- Analyze using RDD 
- Analyze using Dataframe without temp table 
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)

In [ ]:
#This is so I can remember the index when using the rdd functions
for i,j in enumerate(df_twitter.columns):
    print("Column",i,":",j)

Column 0 : created_at
Column 1 : favorite_count
Column 2 : hashtags
Column 3 : id
Column 4 : in_reply_to_status_id
Column 5 : in_reply_to_user_id_str
Column 6 : location
Column 7 : reply_count
Column 8 : retweet_count
Column 9 : source
Column 10 : user_id_str
Column 11 : user_followers_count
Column 12 : user_created_at
Column 13 : user_friends_count
Column 14 : extracted_source


In [ ]:
# Using RDD
#Use the count and distinct functions
print("Total number of tweets:", rdd_twitter.count(),", from",rdd_twitter.map(lambda x: x[10]).distinct().count(),"unique users")

Total number of tweets: 15894 , from 14094 unique users


In [ ]:
# Using DataFrame
print("Total number of tweets:", df_twitter.count(),", from",df_twitter.select("user_id_str").distinct().count(),"unique users")

Total number of tweets: 15894 , from 14094 unique users


In [ ]:
# Using spark.sql and the temporay table.

spark.sql('''
    SELECT COUNT(DISTINCT user_id_str) AS Total_number_of_unique_users
    FROM twitter''').show()


+----------------------------+
|Total_number_of_unique_users|
+----------------------------+
|                       14094|
+----------------------------+



#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)

In [ ]:
# Using RDD

#First get the unique users
userCount = rdd_twitter.map(lambda x: x[10]).distinct()
#And then countByValue
mapping = rdd_twitter.map(lambda x: x[10]).countByValue()
#So we can now filter and count the rows
userCount.filter(lambda x: mapping[x]>1).count()

1016

In [ ]:
# Using DataFrame
df_twitter.groupby("user_id_str").count().filter("count > 1").count()

1016

In [ ]:
# Using spark.sql and the temporay table.
spark.sql('''
    SELECT COUNT(*) AS Users_more_1_tweet FROM (
        SELECT COUNT(user_id_str) FROM twitter
        GROUP BY user_id_str 
        HAVING COUNT(user_id_str) > 1)''').show()

+------------------+
|Users_more_1_tweet|
+------------------+
|              1016|
+------------------+



#### 2.3 Get total number unique extracted_source (1 point each)

In [ ]:
# Using RDD
rdd_twitter.map(lambda x: x[14]).distinct().count()

133

In [ ]:
# Using DataFrame
df_twitter.select("extracted_source").distinct().count()

133

In [ ]:
# Using spark.sql and the temporay table.
spark.sql('''
    SELECT COUNT (DISTINCT extracted_source) AS Unique_Extracted_Source
    FROM twitter''').show()

+-----------------------+
|Unique_Extracted_Source|
+-----------------------+
|                    133|
+-----------------------+



#### 2.4 Get top 5 most used extracted_source

In [ ]:
# Using RDD (5 points)

#Distinct extracted source
ES = rdd_twitter.map(lambda x: x[14]).distinct()
#Counting
mapping = rdd_twitter.map(lambda x: x[14]).countByValue()
#Now we can map using both just taking the 5 most used
ES.takeOrdered(5,key=lambda x:-mapping[x])

['Twitter for Android',
 'Twitter for iPhone',
 'Twitter Web App',
 'Twitter for iPad',
 'Twitter Web Client']

In [ ]:
# Using DataFrame (2 points)
df_twitter.groupby("extracted_source").count().sort(col("count").desc()).show(5)

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+
only showing top 5 rows



In [ ]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('''
        SELECT extracted_source, COUNT(extracted_source) AS Count_Extracted
        FROM twitter
        GROUP BY extracted_source 
        ORDER BY Count_Extracted DESC
        LIMIT 5''').show()

+-------------------+---------------+
|   extracted_source|Count_Extracted|
+-------------------+---------------+
|Twitter for Android|           6262|
| Twitter for iPhone|           5698|
|    Twitter Web App|           2878|
|   Twitter for iPad|            428|
| Twitter Web Client|            136|
+-------------------+---------------+



#### 2.5 Get count of distinct hastags used ( 5 point each) 

In [ ]:
import pyspark.sql.functions as F

In [ ]:
# Using RDD

#flatmap allows me to use the list of hashtags
rdd_twitter.flatMap(lambda x: x[2]).distinct().count()

1215

In [ ]:
# Using DataFrame
df_twitter.select(F.explode(col('hashtags'))).distinct().count()

1215

In [ ]:
# Using spark.sql and the temporay table.
spark.sql('''
        SELECT COUNT(*) AS N_Unique_Hashtags FROM (SELECT DISTINCT EXPLODE(hashtags)
        FROM twitter)''').show()


+-----------------+
|N_Unique_Hashtags|
+-----------------+
|             1215|
+-----------------+



#### 2.6 Get top 5 hashtags

In [ ]:
# Using RDD (4 points)

#The explode function takes the list and "unlist" it
df_twitter.select(F.explode(col('hashtags'))).groupby("col").count().sort(col("count").desc()).show(5)

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# Using DataFrame (2 points)
df_twitter.select(F.explode(col('hashtags'))).groupby("col").count().sort(col("count").desc()).show(5)

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('''SELECT hashtag, COUNT(hashtag) as Hashtag_Count FROM (SELECT EXPLODE(hashtags) as hashtag
        FROM twitter)
        GROUP BY hashtag
        ORDER BY Hashtag_Count DESC
        LIMIT 5''').show()

+--------------------+-------------+
|             hashtag|Hashtag_Count|
+--------------------+-------------+
|طبق_القدرات_للثان...|          385|
|              Corona|          319|
|            OilPrice|          251|
|             COVID19|          125|
|              corona|          123|
+--------------------+-------------+



#### 2.6 Get total number of tweets which are retweeted more than 100 times

In [ ]:
# Using RDD
rdd_twitter.filter(lambda x: x[8]>100).count()

15753

In [ ]:
# Using DataFrame
df_twitter.filter(df_twitter.retweet_count > 100).count()

15753

In [ ]:
# Using spark.sql and the temporay table.
spark.sql('''
        SELECT COUNT(*) AS N_Tweets_More_100 FROM (
            SELECT retweet_count 
            FROM twitter
            WHERE retweet_count > 100)''').show()


+-----------------+
|N_Tweets_More_100|
+-----------------+
|            15753|
+-----------------+



#### 2.8 Get top 3 most retweeted tweets per country (8 points)

In [ ]:
for i,j in enumerate(df_twitter.columns):
    print("Column",i,":",j)

Column 0 : created_at
Column 1 : favorite_count
Column 2 : hashtags
Column 3 : id
Column 4 : in_reply_to_status_id
Column 5 : in_reply_to_user_id_str
Column 6 : location
Column 7 : reply_count
Column 8 : retweet_count
Column 9 : source
Column 10 : user_id_str
Column 11 : user_followers_count
Column 12 : user_created_at
Column 13 : user_friends_count
Column 14 : extracted_source


In [ ]:
import numpy as np
# Using RDD
rdd_twitter.map(lambda x: (x[6],[x[3],x[8]])).groupByKey().mapValues(lambda x: np.sort(list(x),axis=0)[-3:]).collect()

[('India', array([['1252336674580791297', '9973'],
         ['1252336682713649152', '9976'],
         ['1252336686362738689', '9988']], dtype='<U19')),
 ('USA', array([['1252336697825689601', '9982'],
         ['1252336698274562053', '9987'],
         ['1252336700170424322', '9994']], dtype='<U19')),
 ('China', array([['1252336667735863296', '999'],
         ['1252336671917510660', '9993'],
         ['1252336676778545152', '9998']], dtype='<U19')),
 ('Pakistan', array([['1252336687037837312', '9973'],
         ['1252336695778762752', '9975'],
         ['1252336698320658436', '9988']], dtype='<U19')),
 ('Mexico', array([['1252336683644747776', '9971'],
         ['1252336690284204032', '9994'],
         ['1252336699230892040', '9998']], dtype='<U19')),
 ('UK', array([['1252336669778272256', '9985'],
         ['1252336677395197953', '9989'],
         ['1252336683753730057', '9991']], dtype='<U19')),
 ('Spain', array([['1252336686773800963', '9969'],
         ['1252336699121664000', '9981'

In [ ]:
# Using DataFrame
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
#Attain the rank of each row based on retweet count and location with the window,
#Subsequently filter your results to only keep the first 3 values.
df_twitter.select('id','location','retweet_count', rank().over(
    Window().partitionBy("location").orderBy(col("retweet_count").desc()))
    .alias('rank')).orderBy(col("retweet_count").desc()).filter(col('rank') <= 3).sort(col("location"),col("rank").asc()).show()

+-------------------+--------+-------------+----+
|                 id|location|retweet_count|rank|
+-------------------+--------+-------------+----+
|1252335430323888128|  Canada|         9997|   1|
|1252254877939531776|  Canada|         9992|   2|
|1252252082825986051|  Canada|         9987|   3|
|1252253612140490759|   Chile|         9988|   1|
|1252334891951427585|   Chile|         9984|   2|
|1252253710182481920|   Chile|         9978|   3|
|1252335780707684352|   China|         9998|   1|
|1252253596516843520|   China|         9993|   2|
|1252255562525560832|   China|         9984|   3|
|1252334028092399622| Germany|         9999|   1|
|1252330902325248000| Germany|         9997|   2|
|1252252295510855682| Germany|         9990|   3|
|1252332114948874240|   India|         9988|   1|
|1252252336921206787|   India|         9976|   2|
|1252254519116746754|   India|         9973|   3|
|1252252106750377996|   Italy|         9994|   1|
|1252251206027816960|   Italy|         9984|   2|


In [ ]:
# Using spark.sql and the temporay table.

spark.sql('''SELECT id, location, retweet_count, rank
FROM (SELECT id, location, retweet_count, dense_rank() OVER (
    PARTITION BY location ORDER BY retweet_count DESC) as rank
    FROM twitter)
    WHERE rank <=3
    ORDER BY location, rank ASC''').show()

+-------------------+--------+-------------+----+
|                 id|location|retweet_count|rank|
+-------------------+--------+-------------+----+
|1252335430323888128|  Canada|         9997|   1|
|1252254877939531776|  Canada|         9992|   2|
|1252252082825986051|  Canada|         9987|   3|
|1252253612140490759|   Chile|         9988|   1|
|1252334891951427585|   Chile|         9984|   2|
|1252253710182481920|   Chile|         9978|   3|
|1252335780707684352|   China|         9998|   1|
|1252253596516843520|   China|         9993|   2|
|1252255562525560832|   China|         9984|   3|
|1252334028092399622| Germany|         9999|   1|
|1252330902325248000| Germany|         9997|   2|
|1252252295510855682| Germany|         9990|   3|
|1252332114948874240|   India|         9988|   1|
|1252252336921206787|   India|         9976|   2|
|1252254519116746754|   India|         9973|   3|
|1252252106750377996|   Italy|         9994|   1|
|1252251206027816960|   Italy|         9984|   2|


#### 2.9 Total number of tweets per country

In [ ]:
# Using RDD (3 points)
rdd_twitter.map(lambda x: x[6]).countByValue()

defaultdict(int,
            {'Canada': 1441,
             'Chile': 1410,
             'China': 1457,
             'Germany': 1426,
             'India': 1480,
             'Italy': 1422,
             'Mexico': 1409,
             'Pakistan': 1470,
             'Spain': 1464,
             'UK': 1376,
             'USA': 1539})

In [ ]:
# Using DataFrame (2 points)
df_twitter.groupby("location").count().sort(col("count")).show()

+--------+-----+
|location|count|
+--------+-----+
|      UK| 1376|
|  Mexico| 1409|
|   Chile| 1410|
|   Italy| 1422|
| Germany| 1426|
|  Canada| 1441|
|   China| 1457|
|   Spain| 1464|
|Pakistan| 1470|
|   India| 1480|
|     USA| 1539|
+--------+-----+



In [ ]:
# Using spark.sql and the temporay table. (1 point)
spark.sql('''SELECT location, count(id)
FROM twitter
GROUP BY location
ORDER BY count(id)''').show()

+--------+---------+
|location|count(id)|
+--------+---------+
|      UK|     1376|
|  Mexico|     1409|
|   Chile|     1410|
|   Italy|     1422|
| Germany|     1426|
|  Canada|     1441|
|   China|     1457|
|   Spain|     1464|
|Pakistan|     1470|
|   India|     1480|
|     USA|     1539|
+--------+---------+



## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [ ]:
# Using DataFrame

#I need to cast the only column that has arrays (hashtags) to string. Then I can save as CSV.
#In order to save my dataframe grouped by country in different folders, I use the repartition() and partitionBy functions.

df_twitter.withColumn("hashtags", col("hashtags").cast("string")).repartition('location').write.partitionBy('location').csv("/content/saved_twitter_data",header = 'true')

In [ ]:
#Let's check if everything is saved successfully.

#!!! The file name changes everytime you execute the past code. Change it in order to read the csv file.!!!

check_df = spark.read.csv("saved_twitter_data/location=UK/part-00132-7c56631a-b012-4d72-9256-eb530532aec4.c000.csv",header='True')
check_df.show(2)

+--------------------+--------------+--------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+--------------------+--------------------+------------------+-------------------+
|          created_at|favorite_count|hashtags|                 id|in_reply_to_status_id|in_reply_to_user_id_str|reply_count|retweet_count|              source|        user_id_str|user_followers_count|     user_created_at|user_friends_count|   extracted_source|
+--------------------+--------------+--------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+--------------------+--------------------+------------------+-------------------+
|Mon Apr 20 15:01:...|          7682|      []|1252251164256555009|                 null|                   null|        418|         6513|<a href="http://t...|          346443880|                 208|Mon Aug 01 08:15:

#### 3.2 Save the data as parquet files (1 points)

In [ ]:
# Using DataFrame
df_twitter.repartition('location').write.partitionBy('location').parquet("twitter_location.as.parquet")

In [ ]:
#Let's check if everything is saved successfully.
parquetFile = spark.read.parquet("twitter_location.as.parquet")

# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
spark.sql("SELECT * FROM parquetFile").show()

+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+--------------------+--------------------+------------------+-------------------+--------+
|          created_at|favorite_count|            hashtags|                 id|in_reply_to_status_id|in_reply_to_user_id_str|reply_count|retweet_count|              source|        user_id_str|user_followers_count|     user_created_at|user_friends_count|   extracted_source|location|
+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+--------------------+--------------------+------------------+-------------------+--------+
|Mon Apr 20 15:01:...|          5209|                  []|1252251170057457665|                 null|                   null|       6258|         7927|<a h